### Chest Xray

In [3]:
import os
import glob
from tqdm.notebook import tqdm
import numpy as np

In [4]:
import cv2

In [5]:
treinonormal= list(os.listdir('train/NORMAL'))

In [6]:
treinopneu= list(os.listdir('train/PNEUMONIA'))

In [7]:
testenormal=list((os.listdir('test/NORMAL')))

In [8]:
testepneu=list((os.listdir('test/PNEUMONIA')))

In [9]:
x_train = []
y_train = []
x_test = []
y_test = []

In [10]:
for im in treinopneu:
    nome_arquivo = 'train/PNEUMONIA/'+im
    img =  cv2.imread(nome_arquivo, cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(img, (128, 128))
    resized_image = resized_image/255
    x_train.append(resized_image)
    y_train.append(1)

In [11]:
for im in treinonormal:
    nome_arquivo = 'train/NORMAL/'+im
    img =  cv2.imread(nome_arquivo, cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(img, (128, 128))
    resized_image = resized_image/255
    x_train.append(resized_image)
    y_train.append(0)

In [12]:
for im in testenormal:
    nome_arquivo = 'test/NORMAL/'+im
    img =  cv2.imread(nome_arquivo, cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(img, (128, 128))
    resized_image = resized_image/255
    x_test.append(resized_image)
    y_test.append(0)

In [13]:
for im in testepneu:
    nome_arquivo = 'test/PNEUMONIA/'+im
    img =  cv2.imread(nome_arquivo, cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(img, (128, 128))
    resized_image = resized_image/255
    x_test.append(resized_image)
    y_test.append(1)

In [14]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [15]:
model = Sequential()

In [16]:
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [17]:
# Compilar o modelo
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 50176)             0

In [20]:
# Treinar o modelo
model.fit(np.array(x_train), np.array(y_train), epochs=5)

Epoch 1/5
164/164 [==============================] - 59s 358ms/step - loss: 0.2166 - accuracy: 0.9067
Epoch 2/5
164/164 [==============================] - 60s 364ms/step - loss: 0.0954 - accuracy: 0.9641
Epoch 3/5
164/164 [==============================] - 58s 352ms/step - loss: 0.0822 - accuracy: 0.9698
Epoch 4/5
164/164 [==============================] - 58s 354ms/step - loss: 0.0594 - accuracy: 0.9774
Epoch 5/5
164/164 [==============================] - 59s 359ms/step - loss: 0.0573 - accuracy: 0.9780


In [21]:
y_pred = model.predict(np.array(x_test))

20/20 [==============================] - 2s 97ms/step


In [26]:
y_test = np.array(y_test)

In [27]:
# Converter as previsões para classes binárias (0 ou 1)
y_pred_classes = np.round(y_pred).astype(int)

# Converter os rótulos de teste para classes binárias
y_test_classes = y_test.astype(int)  # y_test são seus rótulos de teste

In [30]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

In [28]:
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes)
recall = recall_score(y_test_classes, y_pred_classes)
f1 = f1_score(y_test_classes, y_pred_classes)

In [31]:
# Calcular a matriz de confusão
confusion_matrix = confusion_matrix(y_test_classes, y_pred_classes)

# Imprimir as métricas e a matriz de confusão
print("Acurácia: {:.2f}".format(accuracy))
print("Precisão: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1))
print("Matriz de Confusão:")
print(confusion_matrix)

Acurácia: 0.80
Precisão: 0.76
Recall: 0.99
F1-score: 0.86
Matriz de Confusão:
[[113 121]
 [  5 385]]
